In [99]:
import urllib3
from bs4 import BeautifulSoup
import certifi
import pandas as pd
import pickle
import requests
import json
import numpy as np
import time
import os
from collections import Counter
import subprocess

In [24]:
args = ['sudo', 'service', 'tor','restart']
subprocess.call(args)

1

In [26]:
import wget
import requests
from bs4 import BeautifulSoup

# use Tor
local_proxy = 'socks5://127.1:9050'
socks_proxy = {
    'http': local_proxy,
    'https': local_proxy
}
current_ip = requests.get(
    url='http://icanhazip.com/'
)
print('original IP: ', current_ip.content)
current_ip = requests.get(
    url='http://icanhazip.com/',
    proxies=socks_proxy
)
print('Tor IP: ', current_ip.content)

original IP:  b'133.9.8.123\n'
Tor IP:  b'107.155.49.126\n'


# data scrape

In [2]:
years = range(1992, 2019)

In [3]:
columns = ['japanese', 'english']

In [4]:
data = pd.DataFrame(columns=columns)

In [5]:
def getdata(year):
    url = 'http://www7a.biglobe.ne.jp/~soramimiupdate/past/' + str(year) + 'main.htm'
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, 'html.parser')
    return soup

In [ ]:
for year in years:
    soup = getdata(year)
    table = 1
    while table < 200:
        if soup.select_one("#table" + str(table) + " > tr:nth-child(2) > td:nth-child(3)")==None:
            table = table + 1
        else:
            a = soup.select_one("#table" + str(table) + " > tr:nth-child(2) > td:nth-child(3)").text
            b = soup.select_one("#table" + str(table) + "> tr:nth-child(2) > td:nth-child(4)").text
            df2 = pd.DataFrame(data = [[a, b]], columns = columns)
            data= data.append(df2, ignore_index = True)
            table = table + 1

In [5]:
for i in range(data.shape[0]):
    tmp = data['japanese'][i].replace('\u3000', '')
    tmp = tmp.replace("（☆）", '')
    tmp = tmp.replace("\n", '')
    tmp = tmp.replace("\t", '')
    data['japanese'][i] = tmp

In [6]:
for i in range(data.shape[0]):
    tmp = data['english'][i].replace("\n", '')
    tmp = tmp.replace("\t", '')
    data['english'][i] = tmp

In [8]:
with open('data_refined.pickle', 'wb') as f:
    pickle.dump(data, f)

In [9]:
with open('data_refined.pickle', 'rb') as f:
    data = pickle.load(f)

In [106]:
data2 = data

In [16]:
def kata_to_hira(strj):
    return "".join([chr(ord(ch) - 96) if ("ァ" <= ch <= "ヴ") else ch for ch in strj])

In [17]:
def kanji_to_hira(str):
    url = 'https://hogehoge.tk/nihongo/'
    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())
    headers = {
        'sentence': str,
    }
    r = http.request('POST', url, headers)
    soup = BeautifulSoup(r.data, 'html.parser')
    hira = soup.select_one('body > div > div:nth-child(5) > div')
    if(hira): hira = hira.text
    return hira

In [110]:
for i in range(len(data['japanese'])):
    tmp = kanji_to_hira(data['japanese'][i])
    if(tmp):
        tmp = tmp.replace(' ', '')
        tmp = tmp.replace("\n", '')
        tmp = tmp.replace("\r", '')
        tmp = tmp.replace("〜", "ー")
        tmp = tmp.replace("！", '')
        tmp = tmp.replace("？", '')
        tmp = tmp.replace("!", '')
        tmp = tmp.replace("?", '')
        tmp = tmp.replace("・", '')
        tmp = tmp.replace(".", '')
        tmp = kata_to_hira(tmp)
        data2['japanese'][i] = tmp

In [29]:
data2.to_csv('data_preprocessed_by_hand_add_ipa.csv')

# data load

In [9]:
with open('data_refined.pickle', 'rb') as f:
    data = pickle.load(f)

In [4]:
data2 = pd.read_csv('data_preprocessed_by_hand_add_ipa.csv')

In [97]:
with open('data_ipa_eng_tochu.pickle', 'rb') as f:
    data_ipa = pickle.load(f)

## IPA

#### English

In [9]:
def post_en_ipa(stre, socks_proxy):
    #time.sleep(1)
    r = requests.post("https://tophonetics.com/ja/", {"text_to_transcribe":stre}, proxies=socks_proxy)
    tmp = BeautifulSoup(r.text, 'html.parser')
    tmp = tmp.select_one("#transcr_output").text
    tmp = tmp.replace('ˈ', '')
    return tmp

In [27]:
for en in range(930,len(data2['english'])):
    tmp = post_en_ipa(data2['english'][en], socks_proxy)
    data2['english_ipa'][en] = tmp
    if(tmp == ''):
        print(en)
        break

/usr/local/var/pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
data3 = data2

In [42]:
drop = []
for en in range(len(data2['english'])):
    if(data2['english_ipa'][en].find('-') != -1):
        drop.append(en)

In [44]:
len(drop)

225

In [46]:
data3 = data3.drop(drop)

In [65]:
data3.to_csv('data_preprocessed_by_hand_add_ipa_dropped.csv')

In [101]:
data3 = pd.read_csv('data_preprocessed_by_hand_add_ipa_dropped.csv')

In [102]:
data3 = data3.reset_index()

In [104]:
data3.shape

(776, 8)

## IPAを格納

In [113]:
ipa_e = ['iː', 'ɑː', 'ɔː', 'uː', 'ɜ:', 'eɪ', 'aɪ', 'ɔi', 'aʊ', 'əʊ', 'ɪə', 'eə', 'ʊə', 'tʃ', 'dʒ', 'ə', 'ɪ', 'e', 'æ', 'ɒ', 'ʌ', 'ʊ', 'p', 'b', 't', 'd', 'k', 'g', 'f', 'v', 'θ', 'ð', 's', 'z', 'ʃ', 'ʒ', 'h', 'm', 'n', 'ŋ', 'l', 'r', 'j', 'w']


In [115]:
len(ipa_e)

44

In [116]:
ipa_j = ['a', 'i', 'ɯ', 'e', 'o', 'k', 'g', 'ŋ', 's', 'ʃ', 'ʒ', 'z', 't', 'd', 'n', 'ɲ', 'h', 'ç', 'ɸ', 'b', 'p', 'm', 'j', 'r', 'w', 'ɴ', 'ː']


In [117]:
len(ipa_j)

27

In [107]:
dict_jap = {'ぎゃ':'gja', 'ぎゅ':'gjɯ', 'ぎょ': 'gjo', 'じゃ': 'dʒa', 'じゅ': 'dʒɯ','じょ':'dʒo', 'びゃ': 'bja', 'びゅ':'bjɯ' ,'びょ': 'bjo', 'ぴゃ': 'pja', 'ぴゅ': 'pjɯ', 'ぴょ':'pjo', 'きゃ':'kja', 'きゅ':'kjɯ', 'きょ':'kjo', 'しゃ':'ʃa', 'しゅ':'ʃɯ', 'しょ':'ʃo', 'ちゃ':'tʃa', 'ちゅ':'tʃɯ', 'ちょ':'tʃo', 'にゃ':'ɲa', 'にゅ':'ɲɯ', 'にょ':'ɲo', 'ひゃ':'çja', 'ひゅ':'çjɯ', 'ひょ':'çjo', 'みゃ': 'mja', 'みゅ': 'mjɯ', 'みょ':'mjo', 'りゃ':'rja', 'りゅ': 'rjɯ', 'りょ':'rjo', 'が':'ga', 'ぎ':'gi', 'ぐ':'gɯ', 'げ':'ge', 'ご':'go', 'ざ':'dza', 'じ':'dʒi', 'ず':'dzɯ', 'ぜ':'dze', 'ぞ':'dzo', 'だ':'da', 'で':'de', 'ど':'do', 'ば':'ba', 'び':'bi', 'ぶ':'bɯ', 'べ':'be', 'ぼ':'bo', 'ぱ':'pa', 'ぴ':'pi', 'ぷ':'pɯ', 'ぺ':'pe', 'ぽ':'po', 'あ':'a', 'い':'i', 'う':'ɯ', 'え':'e', 'お':'o', 'か':'ka', 'き':'ki', 'く':'kɯ', 'け':'ke', 'こ':'ko', 'さ':'sa', 'し':'ʃi', 'す':'sɯ', 'せ':'se', 'そ':'so', 'た':'ta', 'ち':'tʃi', 'つ':'tsɯ', 'て':'te','と':'to', 'な':'na', 'に':'ɲi', 'ぬ':'nɯ', 'ね':'ne', 'の':'no', 'は':'ha', 'ひ':'çi', 'ふ':'ɸɯ', 'へ':'he', 'ほ':'ho', 'ま':'ma', 'み':'mi', 'む':'mɯ', 'め':'me', 'も':'mo', 'や':'ja', 'ゆ':'jɯ', 'よ':'jo', 'ら':'ra', 'り':'ri', 'る':'rɯ', 'れ':'re', 'ろ':'ro', 'わ':'wa', 'を':'o', 'ん':'ɴ', 'ー':'ː'}

### IPAデータ

In [108]:
N = len(data3['english'])

In [109]:
data_ipa = np.zeros((len(ipa_e) + len(ipa_j), N))
data_ipa = pd.DataFrame(data_ipa, index=ipa_e+ipa_j)

In [110]:
for en in range(len(data3['english'])):
    tmp = data3['english_ipa'][en]
    for i in range(len(ipa_e)):
        if tmp.count(ipa_e[i]) != 0:
            data_ipa[en][i] = tmp.count(ipa_e[i])
            tmp = tmp.replace(ipa_e[i], '')
        else:
            data_ipa[en][i] = 0

#### Japanese

In [111]:
for ja in range(len(data3['japanese'])):
    n = 0
    tmp = data3['japanese'][ja]
    for i in list(dict_jap.keys()):
        if tmp.find(i) != -1:
            for w in dict_jap[i]:
                data_ipa[ja][len(ipa_e) + ipa_j.index(w)] = data_ipa[ja][len(ipa_e) + ipa_j.index(w)] + tmp.count(i)
            tmp = tmp.replace(i, '')
        n = n + 1

In [112]:
data_ipa.to_csv('ipa_matrix.csv')